In [13]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd

# Data

In [14]:
data_dir = Path('T:/data/base_2022/CVM/Calib_CVM_B/')
data_dir

WindowsPath('T:/data/base_2022/CVM/Calib_CVM_B')

In [15]:
cvm_zonal_properties = data_dir/'model_output/Zonal Properties CVM.csv'

employment = (
    pd.read_csv(
        cvm_zonal_properties,
        usecols=[
            'TAZ', 'CVM_IN', 'CVM_RE', 'CVM_SV', 'CVM_TH', 'CVM_WH', 'CVM_WH', 'CVM_GO',
        ],
    )
    .rename(
        columns={
            'CVM_IN': 'Industrial',
            'CVM_RE': 'Retail',
            'CVM_SV': 'Service',
            'CVM_TH': 'Transport',
            'CVM_WH': 'Wholesale',
            'CVM_GO': 'Government/Office',
        }
    )
    .melt(id_vars='TAZ', var_name='industry', value_name='employment')
    .assign(
        industry=lambda df: (
            df.industry
            .map(
                {
                    'Industrial': 'Industrial',
                    'Retail': 'Retail',
                    'Service': 'Service/Gov/Office/FA',
                    'Transport': 'Transport',
                    'Wholesale': 'Wholesale',
                    'Government/Office': 'Service/Gov/Office/FA',
                }
            )
            .astype(
                pd.CategoricalDtype(
                    categories=[
                        'Industrial',
                        'Wholesale',
                        'Retail',
                        'Service/Gov/Office/FA',
                        'Transport',
                    ],
                    ordered=True,
                )
            )
        )
    )
    .set_index('TAZ')
    .groupby('industry')
    .sum()
)
employment

,employment
industry,
Industrial,270020.82
Wholesale,163136.90
Retail,183812.30
Service/Gov/Office/FA,1346153.73
Transport,44556.26


In [16]:
cvm_tours_path = data_dir/'model_output/commercialVehicleTours.csv'

tours = (
    pd.read_csv(
        cvm_tours_path,
        usecols=['tourID', 'actorType', 'departTimeFiveTod', 'tourMode'],
    )
    .assign(
        industry=lambda df: df.actorType
        .map(
            {
                'Government\\Office': 'Service/Gov/Office/FA',
                'Service': 'Service/Gov/Office/FA',
                'Fleet Allocator': 'Service/Gov/Office/FA',
                'Industry': 'Industrial',
                'Retail': 'Retail',
                'Transport': 'Transport',
                'Wholesale': 'Wholesale',
            }
        )
        .astype(
            pd.CategoricalDtype(
                categories=[
                    'Industrial',
                    'Wholesale',
                    'Retail',
                    'Service/Gov/Office/FA',
                    'Transport',
                ],
                ordered=True,
            )
        )
    )
    .drop(columns=['actorType'])
    .set_index('tourID')
)
tours.head(2)

,departTimeFiveTod,tourMode,industry
tourID,,,
1,3.0,Light Heavy Duty Truck,Service/Gov/Office/FA
2,3.0,Drive Alone,Service/Gov/Office/FA


In [17]:
cvm_trips_path = data_dir/'model_output/commercialVehicleTrips.csv'

trips = (
    pd.read_csv(
        cvm_trips_path,
        usecols=['tripID', 'tourID', 'stopID', 'weightTrip', 'distanceTotal', 'departTimeFiveTod', 'tripMode'],
    )
    .assign(
        tod=lambda df: df.departTimeFiveTod
        .map(
            {
                1: 'OE (3am - 6am)',
                2: 'AM (6am - 9am)',
                3: 'MD (9am - 3:30pm)',
                4: 'PM (3:30pm - 7pm)',
                5: 'OL (7pm - 3am)',
            }
        )
        .astype(
            pd.CategoricalDtype(
                categories=[
                    'OE (3am - 6am)',
                    'AM (6am - 9am)',
                    'MD (9am - 3:30pm)',
                    'PM (3:30pm - 7pm)',
                    'OL (7pm - 3am)',
                ],
                ordered=True,
            )
        )
    )
    .assign(
        mode=lambda df: df.tripMode
        .map(
            {
                'Drive Alone': 'DA',
                'Light Heavy Duty Truck': 'LHD',
                'Medium Heavy Duty Truck': 'MHD',
                'Heavy Heavy Duty Truck': 'HHD',
            }
        )
        .astype(
            pd.CategoricalDtype(
                categories=[
                    'DA',
                    'LHD',
                    'MHD',
                    'HHD',
                ],
                ordered=True,
            )
        )
    )
    .drop(columns=['departTimeFiveTod', 'tripMode'])
    .set_index(['tripID', 'tourID', 'stopID'])
)
trips.head(2)

,,,weightTrip,distanceTotal,tod,mode
tripID,tourID,stopID,,,,
1,1,1,3.36,54.307570,MD (9am - 3:30pm),DA
2,1,2,3.36,6.767837,MD (9am - 3:30pm),DA


In [18]:
merged = (
    trips.reset_index()
    .merge(tours.reset_index(), on='tourID', how='left')
    .set_index(['tripID', 'tourID', 'stopID'])
    .assign(vmt=lambda df: df.weightTrip * df.distanceTotal)
)
merged.head(2)

,,,weightTrip,distanceTotal,tod,mode,departTimeFiveTod,tourMode,industry,vmt
tripID,tourID,stopID,,,,,,,,
1,1,1,3.36,54.307570,MD (9am - 3:30pm),DA,3.0,Light Heavy Duty Truck,Service/Gov/Office/FA,182.473435
2,1,2,3.36,6.767837,MD (9am - 3:30pm),DA,3.0,Light Heavy Duty Truck,Service/Gov/Office/FA,22.739932


# Summaries

### Table 15

In [19]:
table_15_data = pd.DataFrame(
    {
        'Employees': employment.groupby('industry').employment.sum(),
        'Tours': merged.groupby(['industry', 'tourID']).weightTrip.max().to_frame().groupby('industry').weightTrip.sum(),
        'Trips': merged.groupby('industry').weightTrip.sum(),
        'Trips_DA': merged.query("`mode` == 'DA'").groupby('industry').weightTrip.sum(),
        'Trips_LHD': merged.query("`mode` == 'LHD'").groupby('industry').weightTrip.sum(),
        'Trips_MHD': merged.query("`mode` == 'MHD'").groupby('industry').weightTrip.sum(),
        'Trips_HHD': merged.query("`mode` == 'HHD'").groupby('industry').weightTrip.sum(),
        'VMT': merged.groupby('industry').vmt.sum(),
        'VMT_DA': merged.query("`mode` == 'DA'").groupby('industry').vmt.sum(),
        'VMT_LHD': merged.query("`mode` == 'LHD'").groupby('industry').vmt.sum(),
        'VMT_MHD': merged.query("`mode` == 'MHD'").groupby('industry').vmt.sum(),
        'VMT_HHD': merged.query("`mode` == 'HHD'").groupby('industry').vmt.sum(),
    }
)
table_15_data.reset_index().set_index('industry').to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_table15.csv')
table_15_data


,Employees,Tours,Trips,Trips_DA,Trips_LHD,Trips_MHD,Trips_HHD,VMT,VMT_DA,VMT_LHD,VMT_MHD,VMT_HHD
industry,,,,,,,,,,,,
Industrial,270020.82,70798.0,233095.5,183832.80,18863.30,3722.40,26677.0,1.909252e+06,1.392734e+06,1.785404e+05,43423.189543,294553.778902
Wholesale,163136.90,31473.0,170674.5,134856.48,21347.78,6114.24,8356.0,1.853424e+06,1.412388e+06,2.402627e+05,77842.216808,122931.625297
Retail,183812.30,65584.0,289123.0,246736.32,30777.24,6913.44,4696.0,2.131067e+06,1.796477e+06,2.330378e+05,56528.997759,45022.601813
Service/Gov/Office/FA,1346153.73,260176.0,1196543.5,876046.56,169643.38,65131.56,85722.0,9.516155e+06,6.707129e+06,1.418204e+06,563558.144541,827263.425319
Transport,44556.26,10759.0,43905.0,28993.92,5239.08,1701.00,7971.0,2.809321e+05,1.741652e+05,3.815557e+04,13361.620285,55249.661319


### Figure 9

In [20]:
fig_9_data = ( # The time-of-day distribution of weighted tours for each industry
    (
        ( # Total weighted tours for each industry, for each time of day
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'tod', 'tourID']).max()
            .groupby(['industry', 'tod'])
            .weightTrip.sum()
        ) 
        / ( # Total weighted tours for each industry
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'tod', 'tourID']).max()
            .groupby(['industry'])
            .weightTrip.sum()
        )
    )
    .to_frame()
    .reset_index()
    .pivot(columns='tod', index='industry', values='weightTrip')
)
fig_9_data.to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_figure9.csv')
fig_9_data

tod,OE (3am - 6am),AM (6am - 9am),MD (9am - 3:30pm),PM (3:30pm - 7pm),OL (7pm - 3am)
industry,,,,,
Industrial,0.266618,0.419051,0.303229,0.009732,0.001370
Wholesale,0.223779,0.432069,0.336971,0.006514,0.000667
Retail,0.114449,0.391940,0.478913,0.011009,0.003690
Service/Gov/Office/FA,0.160876,0.404196,0.419743,0.012849,0.002337
Transport,0.323915,0.441956,0.222047,0.006041,0.006041


### Figure 10

In [21]:
fig_10_data = ( # The vehicle mode distribution of weighted tours for each industry
    (
        ( # Total weighted tours for each industry, for vehicle mode
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'mode', 'tourID']).max()
            .groupby(['industry', 'mode'])
            .weightTrip.sum()
        ) 
        / ( # Total weighted tours for each industry
            merged.reset_index()
            .drop_duplicates(subset='tourID')
            .groupby(['industry', 'mode', 'tourID']).max()
            .groupby(['industry'])
            .weightTrip.sum()
        )
    )
    .to_frame()
)
fig_10_data.to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_figure10.csv')
fig_10_data

weightTrip
industry              mode            
Industrial            DA      0.789514
                      LHD     0.065304
                      MHD     0.010729
                      HHD     0.134453
Wholesale             DA      0.768628
                      LHD     0.113437
                      MHD     0.032496
                      HHD     0.085438
Retail                DA      0.851798
                      LHD     0.107262
                      MHD     0.024366
                      HHD     0.016574
Service/Gov/Office/FA DA      0.815743
                      LHD     0.100625
                      MHD     0.031286
                      HHD     0.052345
Transport             DA      0.558476
                      LHD     0.119591
                      MHD     0.040587
                      HHD     0.281346

### Figure 12

In [22]:
fig_12_data = ( # The vehicle mode distribution of weighted tours for each industry
    (
        ( # Weighted average trip length for each mode/industry
            (
                merged
                .assign(weighted_distance=lambda df: df.distanceTotal * df.weightTrip)
                .groupby(['mode', 'industry']).weighted_distance.sum()
            )
            / merged.groupby(['mode', 'industry']).weightTrip.sum()
        ) 
    )
    .rename('distance')
    .to_frame()
)
fig_12_data.to_csv('T:/data/base_2022/CVM/Calib_CVM_B/model_figure12.csv')
fig_12_data

distance
mode industry                        
DA   Industrial              7.576093
     Wholesale              10.473266
     Retail                  7.280960
     Service/Gov/Office/FA   7.656133
     Transport               6.006956
LHD  Industrial              9.464963
     Wholesale              11.254693
     Retail                  7.571759
     Service/Gov/Office/FA   8.359914
     Transport               7.282877
MHD  Industrial             11.665374
     Wholesale              12.731299
     Retail                  8.176682
     Service/Gov/Office/FA   8.652612
     Transport               7.855156
HHD  Industrial             11.041488
     Wholesale              14.711779
     Retail                  9.587437
     Service/Gov/Office/FA   9.650538
     Transport               6.931334